In [84]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from pyts.multivariate.classification import MultivariateClassifier
from pyts.classification import TimeSeriesForest
from sklearn.model_selection import GridSearchCV
from joblib import Parallel, delayed
from sklearn.preprocessing import RobustScaler
from mpl_toolkits.mplot3d import Axes3D
import itertools
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
import plotly.graph_objects as go
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import pywt
from sklearn.utils import resample
from scipy.stats import skew, kurtosis

**Loading Partition 1.**

In [127]:
main_folder_path = "/home/afrid/Documents/NASA/dataverse_files/partition1"
folder_name_1 = 'FL'
folder_name_2 = 'NF'
folder_path_1 = os.path.join(main_folder_path, folder_name_1)
folder_path_2 = os.path.join(main_folder_path, folder_name_2)
files_folder_1 = os.listdir(folder_path_1)
files_folder_2 = os.listdir(folder_path_2)

In [128]:
dfs_folder_1 = [pd.read_csv(os.path.join(folder_path_1, file), sep='\t', usecols=[1, 2, 3]).assign(Class=1) for file in files_folder_1]
dfs_folder_2 = [pd.read_csv(os.path.join(folder_path_2, file), sep='\t', usecols=[1, 2, 3]).assign(Class=0) for file in files_folder_2]


In [129]:
data_folder_1 = np.stack([df.values for df in dfs_folder_1], axis=0)
data_folder_2 = np.stack([df.values for df in dfs_folder_2], axis=0)

print("Data types after loading CSV files:")
print(dfs_folder_1[0].dtypes)
print(dfs_folder_2[0].dtypes)

Data types after loading CSV files:
TOTUSJH    float64
TOTBSQ     float64
TOTPOT     float64
Class        int64
dtype: object
TOTUSJH    float64
TOTBSQ     float64
TOTPOT     float64
Class        int64
dtype: object


In [130]:
data_folder_1[:, :, -1] = data_folder_1[:, :, -1].astype(int)
data_folder_2[:, :, -1] = data_folder_2[:, :, -1].astype(int)


print("Data types after conversion:")
print(data_folder_1[:, :, -1].dtype)
print(data_folder_2[:, :, -1].dtype)

Data types after conversion:
float64
float64


In [131]:
final_data = np.concatenate([data_folder_1, data_folder_2], axis=0)

In [132]:
fifth_subarray = data_folder_1[4]

missing_values_present = np.isnan(fifth_subarray).any()

if missing_values_present:
    print("Missing values present in the 5th subarray of data_folder_1.")
else:
    print("No missing values in the 5th subarray of data_folder_1.")


Missing values present in the 5th subarray of data_folder_1.


In [133]:
subarrays_with_missing_values = []
for i, subarray in enumerate(final_data):
    missing_values_present = np.isnan(subarray).any()
    if missing_values_present:
        subarrays_with_missing_values.append(i)

In [134]:
subarrays_with_missing_values = []
for i, subarray in enumerate(final_data):
    subarray = np.array(subarray)
    missing_values_present = np.isnan(subarray)
    if np.any(missing_values_present):
        subarrays_with_missing_values.append(i)
        column_means = np.nanmean(subarray, axis=0)
        for j in range(subarray.shape[1]):
            subarray[:, j][missing_values_present[:, j]] = column_means[j]
        final_data[i] = subarray.tolist()

In [135]:
missing_values_per_array = [
    np.any(np.isnan(array)) for array in final_data
]

In [136]:
data_folder_1.shape

(1254, 60, 4)

In [137]:
data_folder_2.shape

(72238, 60, 4)

In [138]:
final_data.shape

(73492, 60, 4)

In [139]:
final_data

array([[[1.31512064e+03, 1.84249705e+10, 3.01690142e+23, 1.00000000e+00],
        [1.29529138e+03, 1.84362189e+10, 3.02408756e+23, 1.00000000e+00],
        [1.29107235e+03, 1.84317213e+10, 3.01902259e+23, 1.00000000e+00],
        ...,
        [1.26301717e+03, 1.70505430e+10, 2.86850258e+23, 1.00000000e+00],
        [1.25975811e+03, 1.70497188e+10, 2.87663620e+23, 1.00000000e+00],
        [1.26777601e+03, 1.70489327e+10, 2.88869871e+23, 1.00000000e+00]],

       [[1.69928306e+03, 1.89409482e+10, 3.39950428e+23, 1.00000000e+00],
        [1.70093419e+03, 1.88158286e+10, 3.39971390e+23, 1.00000000e+00],
        [1.68612987e+03, 1.87045488e+10, 3.39017681e+23, 1.00000000e+00],
        ...,
        [1.75297986e+03, 1.91046967e+10, 3.47880711e+23, 1.00000000e+00],
        [1.77367458e+03, 1.93368172e+10, 3.52739906e+23, 1.00000000e+00],
        [1.83538591e+03, 1.96835259e+10, 3.58910389e+23, 1.00000000e+00]],

       [[4.09370433e+03, 6.38030309e+10, 1.06127205e+24, 1.00000000e+00],
        

In [140]:
unique, counts = np.unique(final_data[:, 0, -1], return_counts=True)
print(dict(zip(unique, counts)))

{0.0: 72238, 1.0: 1254}


In [154]:
unique_labels, counts = np.unique(final_data[:, 0, -1], return_counts=True)
label_counts = dict(zip(unique_labels, counts))
print("Counts for each class:", label_counts)


indexes = {}
for label in unique_labels:
    indexes[label] = np.where(final_data[:, 0, -1] == label)[0]
    print(f"Indexes for class {label}: {indexes[label]}")

Counts for each class: {0.0: 72238, 1.0: 1254}
Indexes for class 0.0: [ 1254  1255  1256 ... 73489 73490 73491]
Indexes for class 1.0: [   0    1    2 ... 1251 1252 1253]


**Loading Partition 2.**

In [141]:
main_folder_path_p2 = "/home/afrid/Documents/NASA/dataverse_files/partition2"
folder_name_FL_p2 = 'FL'
folder_name_NF_p2 = 'NF'

folder_path_FL_p2 = os.path.join(main_folder_path_p2, folder_name_FL_p2)
folder_path_NF_p2 = os.path.join(main_folder_path_p2, folder_name_NF_p2)
files_FL_p2 = os.listdir(folder_path_FL_p2)
files_NF_p2 = os.listdir(folder_path_NF_p2)


In [142]:
dfs_FL_p2 = [pd.read_csv(os.path.join(folder_path_FL_p2, file), sep='\t', usecols=[1, 2, 3]).assign(Class=1) for file in files_FL_p2]
dfs_NF_p2 = [pd.read_csv(os.path.join(folder_path_NF_p2, file), sep='\t', usecols=[1, 2, 3]).assign(Class=0) for file in files_NF_p2]
data_FL_p2 = np.stack([df.values for df in dfs_FL_p2], axis=0)
data_NF_p2 = np.stack([df.values for df in dfs_NF_p2], axis=0)
data_FL_p2[:, :, -1] = data_FL_p2[:, :, -1].astype(int)
data_NF_p2[:, :, -1] = data_NF_p2[:, :, -1].astype(int)

In [143]:
final_data_p2 = np.concatenate([data_FL_p2, data_NF_p2], axis=0)

In [144]:
subarrays_with_missing_values_NF_p2 = []

for i, subarray in enumerate(final_data_p2):
    subarray_NF_p2 = np.array(subarray)
    missing_values_present_NF_p2 = np.isnan(subarray_NF_p2)

    if np.any(missing_values_present_NF_p2):
        subarrays_with_missing_values_NF_p2.append(i)
        column_means_NF_p2 = np.nanmean(subarray_NF_p2, axis=0)
        for j_NF_p2 in range(subarray_NF_p2.shape[1]):
            subarray_NF_p2[:, j_NF_p2][missing_values_present_NF_p2[:, j_NF_p2]] = column_means_NF_p2[j_NF_p2]
        final_data_p2[i] = subarray_NF_p2.tolist()

sampling.

In [155]:
def undersample_data(X, y):
    dataset = np.hstack((X.reshape(X.shape[0], -1), y.reshape(-1, 1)))
    majority = dataset[dataset[:, -1] == 0.0]
    minority = dataset[dataset[:, -1] == 1.0]
    majority_undersampled = resample(majority, replace=False, n_samples=len(minority), random_state=123)
    dataset_undersampled = np.vstack((minority, majority_undersampled))
    X_undersampled = dataset_undersampled[:, :-1].reshape(-1, X.shape[1], X.shape[2])
    y_undersampled = dataset_undersampled[:, -1]

    return X_undersampled, y_undersampled





normalization.

In [156]:
def robust_normalize_3d(data):
    scaled_subarrays = []
    for subarray in data:
        scaler = RobustScaler()
        scaled_subarray = scaler.fit_transform(subarray[:, :-1])
        scaled_subarray_with_label = np.hstack((scaled_subarray, subarray[:, -1].reshape(-1, 1)))
        scaled_subarrays.append(scaled_subarray_with_label)

    scaled_data = np.array(scaled_subarrays)
    return scaled_data

#normalized_train_data = robust_normalize_3d(final_train_data)

#normalized_train_data.shape

wavelet.

In [157]:
def apply_dwt(data, wavelet='db4', level=3):
    coeffs = pywt.wavedec(data, wavelet=wavelet, level=level, axis=0)
    concatenated = np.concatenate(coeffs, axis=0)
    return concatenated

TSS and HSS.

In [158]:
def custom_scorer(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    if cm.shape == (2, 2):
        tp = cm[1, 1]
        fn = cm[1, 0]
        fp = cm[0, 1]
        tn = cm[0, 0]

        sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tss = sensitivity - fpr

        p = tp + fn
        n = fp + tn
        hss = (2 * ((tp * tn) - (fn * fp))) / ((p * (fn + tn)) + (n * (tp + fp)))

        return tss, hss
    elif cm.shape == (1, 1):
        return 1 if y_true[0] == y_pred[0] else 0, 0
    else:
        print(f"Error: Confusion matrix shape {cm.shape} is not handled.")
        return 0, 0

tss_scorer = make_scorer(lambda y_true, y_pred: custom_scorer(y_true, y_pred)[0], greater_is_better=True)
hss_scorer = make_scorer(lambda y_true, y_pred: custom_scorer(y_true, y_pred)[1], greater_is_better=True)

Model.

In [159]:
clf = MultivariateClassifier(estimator=TimeSeriesForest(random_state=53))

In [160]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
    clf.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    tss, hss = custom_scorer(y_test, y_pred)
    return tss, hss

In [161]:
results = {}

Scenarios.

In [167]:
#Undersampled data (on only training data) + Wavelet Transform
X_train_under, y_train_under = undersample_data(final_data[:, :, :-1], final_data[:, 0, -1])
X_train_under_wave = np.array([apply_dwt(subarray, 'db4', 3) for subarray in X_train_under])
X_test_wave = np.array([apply_dwt(subarray[:, :-1], 'db4', 3) for subarray in final_data_p2])
results['scenario_1'] = evaluate_model(X_train_under_wave, y_train_under, X_test_wave, final_data_p2[:, 0, -1], clf)
results['scenario_1']

(0.7457747889252184, 0.10261476813672603)

In [168]:
#Undersampled data (both on training and testing data) + wavelet transform
X_test_under, y_test_under = undersample_data(final_data_p2[:, :, :-1], final_data_p2[:, 0, -1])
X_test_under_wave = np.array([apply_dwt(subarray, 'db4', 3) for subarray in X_test_under])
results['scenario_2'] = evaluate_model(X_train_under_wave, y_train_under, X_test_under_wave, y_test_under, clf)
results['scenario_2'] 

(0.7489300998573467, 0.7489300998573466)

In [169]:
#Undersampled data (on only training data) + normalization + Wavelet Transform
X_train_under_norm = robust_normalize_3d(X_train_under)
X_train_under_norm_wave = np.array([apply_dwt(subarray, 'db4', 3) for subarray in X_train_under_norm])
results['scenario_3'] = evaluate_model(X_train_under_norm_wave, y_train_under, X_test_wave, final_data_p2[:, 0, -1], clf)
results['scenario_3']


(0.7457747889252184, 0.10261476813672603)

In [170]:
#Undersampled data (both on training and testing data) + normalization + wavelet transform
X_test_under_norm = robust_normalize_3d(X_test_under)
X_test_under_norm_wave = np.array([apply_dwt(subarray, 'db4', 3) for subarray in X_test_under_norm])
results['scenario_4'] = evaluate_model(X_train_under_norm_wave, y_train_under, X_test_under_norm_wave, y_test_under, clf)
results['scenario_4']

(0.7489300998573467, 0.7489300998573466)

In [171]:
for scenario, (tss, hss) in results.items():
    print(f"{scenario}: TSS = {tss}, HSS = {hss}")

scenario_1: TSS = 0.7457747889252184, HSS = 0.10261476813672603
scenario_2: TSS = 0.7489300998573467, HSS = 0.7489300998573466
scenario_3: TSS = 0.7457747889252184, HSS = 0.10261476813672603
scenario_4: TSS = 0.7489300998573467, HSS = 0.7489300998573466


**Sampling.**

partition 1

In [18]:
class_labels = final_data[:, 0, -1]
class_0_indices = np.where(class_labels == 0)[0]
class_1_indices = np.where(class_labels == 1)[0]
selected_class_0_indices = class_0_indices[:len(class_1_indices)]
selected_indices = np.concatenate([class_1_indices, selected_class_0_indices])
final_train_data = final_data[selected_indices]
final_train_data[:, 0, -1] = final_train_data[:, 0, -1].astype(int)

print(final_train_data.shape)

(2508, 60, 4)


partition 2

In [63]:
class_labels_p2 = final_data_p2[:, 0, -1]
class_0_indices_p2 = np.where(class_labels_p2 == 0)[0]
class_1_indices_p2 = np.where(class_labels_p2 == 1)[0]
selected_class_0_indices_p2 = class_0_indices_p2[:len(class_1_indices_p2)]
selected_indices_p2 = np.concatenate([class_1_indices_p2, selected_class_0_indices_p2])
final_data_p2 = final_data_p2[selected_indices_p2]
final_data_p2[:, 0, -1] = final_data_p2[:, 0, -1].astype(int)


print(final_data_p2.shape)

(2804, 60, 4)


In [64]:
final_train_data.shape

(2508, 60, 4)

In [65]:
final_train_data

array([[[1.31512064e+03, 1.84249705e+10, 3.01690142e+23, 1.00000000e+00],
        [1.29529138e+03, 1.84362189e+10, 3.02408756e+23, 1.00000000e+00],
        [1.29107235e+03, 1.84317213e+10, 3.01902259e+23, 1.00000000e+00],
        ...,
        [1.26301717e+03, 1.70505430e+10, 2.86850258e+23, 1.00000000e+00],
        [1.25975811e+03, 1.70497188e+10, 2.87663620e+23, 1.00000000e+00],
        [1.26777601e+03, 1.70489327e+10, 2.88869871e+23, 1.00000000e+00]],

       [[1.69928306e+03, 1.89409482e+10, 3.39950428e+23, 1.00000000e+00],
        [1.70093419e+03, 1.88158286e+10, 3.39971390e+23, 1.00000000e+00],
        [1.68612987e+03, 1.87045488e+10, 3.39017681e+23, 1.00000000e+00],
        ...,
        [1.75297986e+03, 1.91046967e+10, 3.47880711e+23, 1.00000000e+00],
        [1.77367458e+03, 1.93368172e+10, 3.52739906e+23, 1.00000000e+00],
        [1.83538591e+03, 1.96835259e+10, 3.58910389e+23, 1.00000000e+00]],

       [[4.09370433e+03, 6.38030309e+10, 1.06127205e+24, 1.00000000e+00],
        

In [66]:
final_data_p2

array([[[2.10433562e+03, 1.66309802e+10, 2.44271097e+23, 1.00000000e+00],
        [2.13771943e+03, 1.66821080e+10, 2.43289777e+23, 1.00000000e+00],
        [2.13605186e+03, 1.67190788e+10, 2.43625786e+23, 1.00000000e+00],
        ...,
        [2.17481045e+03, 1.74546880e+10, 2.90306262e+23, 1.00000000e+00],
        [2.14550720e+03, 1.74297430e+10, 2.90576637e+23, 1.00000000e+00],
        [2.11443359e+03, 1.73691727e+10, 2.84922323e+23, 1.00000000e+00]],

       [[4.46694782e+03, 6.97694787e+10, 9.63171118e+23, 1.00000000e+00],
        [4.49201943e+03, 7.00970991e+10, 9.72063961e+23, 1.00000000e+00],
        [4.56087991e+03, 7.02730264e+10, 9.72636252e+23, 1.00000000e+00],
        ...,
        [4.42512551e+03, 6.63204200e+10, 9.61936558e+23, 1.00000000e+00],
        [4.40896833e+03, 6.62384349e+10, 9.59023923e+23, 1.00000000e+00],
        [4.41834384e+03, 6.60643521e+10, 9.58056537e+23, 1.00000000e+00]],

       [[2.31663333e+03, 2.96306831e+10, 3.42228002e+23, 1.00000000e+00],
        

**Normalization.**

In [83]:
def robust_normalize_3d(data):
    """
    Apply RobustScaler to each 2D subarray in a 3D array.
    
    Parameters:
    - data: 3D numpy array, shape (n_samples, n_timepoints, n_features)
    
    Returns:
    - scaled_data: 3D numpy array with the same shape, where each 2D subarray has been scaled.
    """
 
    scaled_subarrays = []
    for subarray in data:
        scaler = RobustScaler()
        scaled_subarray = scaler.fit_transform(subarray[:, :-1])
        scaled_subarray_with_label = np.hstack((scaled_subarray, subarray[:, -1].reshape(-1, 1)))
        scaled_subarrays.append(scaled_subarray_with_label)

    scaled_data = np.array(scaled_subarrays)
    return scaled_data

normalized_train_data = robust_normalize_3d(final_train_data)

normalized_train_data.shape

(2508, 60, 4)

Model Example.

In [67]:

def apply_dwt(data, wavelet='db4', level=3):
    coeffs = pywt.wavedec(data, wavelet=wavelet, level=level, axis=0)
    concatenated = np.concatenate(coeffs, axis=0)
    return concatenated

In [68]:
transformed_train_data = np.array([apply_dwt(subarray[:, :-1], 'db4', 3) for subarray in final_train_data])

In [69]:
transformed_train_data.shape

(2508, 79, 3)

In [70]:
transformed_test_data = np.array([apply_dwt(subarray[:, :-1], 'db4', 3) for subarray in final_data_p2])


In [71]:
clf = MultivariateClassifier(estimator=TimeSeriesForest(random_state=53))


In [72]:
train_labels = final_train_data[:, 0, -1]
train_labels 

array([1., 1., 1., ..., 0., 0., 0.])

In [73]:
unique_labels, counts = np.unique(train_labels, return_counts=True)
label_counts = dict(zip(unique_labels, counts))
print("Counts for each class:", label_counts)


indexes = {}
for label in unique_labels:
    indexes[label] = np.where(train_labels == label)[0]
    print(f"Indexes for class {label}: {indexes[label]}")


Counts for each class: {0.0: 1254, 1.0: 1254}
Indexes for class 0.0: [1254 1255 1256 ... 2505 2506 2507]
Indexes for class 1.0: [   0    1    2 ... 1251 1252 1253]


In [74]:
clf.fit(transformed_train_data, train_labels)


MultivariateClassifier(estimator=TimeSeriesForest(random_state=53))

In [75]:
test_labels = final_data_p2[:, 0, -1] 

In [76]:

unique_labels, counts = np.unique(test_labels, return_counts=True)
label_counts = dict(zip(unique_labels, counts))
print("Counts for each class:", label_counts)

indexes = {}
for label in unique_labels:
    indexes[label] = np.where(test_labels == label)[0]
    print(f"Indexes for class {label}: {indexes[label]}")

Counts for each class: {0.0: 1402, 1.0: 1402}
Indexes for class 0.0: [1402 1403 1404 ... 2801 2802 2803]
Indexes for class 1.0: [   0    1    2 ... 1399 1400 1401]


In [77]:
predictions = clf.predict(transformed_test_data)

In [78]:
def custom_scorer(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    if cm.shape == (2, 2):
        tp = cm[1, 1]
        fn = cm[1, 0]
        fp = cm[0, 1]
        tn = cm[0, 0]

        sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tss = sensitivity - fpr

        p = tp + fn
        n = fp + tn
        hss = (2 * ((tp * tn) - (fn * fp))) / ((p * (fn + tn)) + (n * (tp + fp)))

        return tss, hss
    elif cm.shape == (1, 1):
        return 1 if y_true[0] == y_pred[0] else 0, 0
    else:
        print(f"Error: Confusion matrix shape {cm.shape} is not handled.")
        return 0, 0

tss_scorer = make_scorer(lambda y_true, y_pred: custom_scorer(y_true, y_pred)[0], greater_is_better=True)
hss_scorer = make_scorer(lambda y_true, y_pred: custom_scorer(y_true, y_pred)[1], greater_is_better=True)



In [79]:
tss, hss = custom_scorer(test_labels, predictions)

print(f"TSS: {tss}")
print(f"HSS: {hss}")

TSS: 0.7503566333808844
HSS: 0.7503566333808844
